In [1]:
%%bash

chmod 777 /tmp
mkdir data
apt-get update --allow-unauthenticated 
apt-get update -y --fix-missing 
pip install selenium
apt-get install chromium-chromedriver -y --fix-missing
pip install joblib
apt-get update --fix-missing


Couldn't find program: 'bash'


In [2]:
# ! ls /usr/bin # Webdriver and Chromium binary installed here

In [3]:
## Import list of https proxies
import warnings;
warnings.filterwarnings('ignore')

import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
URL = "https://free-proxy-list.net/"
req = requests.get(URL, headers = headers) # .json()
soup = BeautifulSoup(req.text, "lxml")

for body in soup("tbody"):
    body.unwrap()

df = pd.read_html(str(soup), flavor="bs4")
df = pd.DataFrame(df[0])
proxies = df[(df.Https == 'yes')] # (df.Https == 'yes') & (df.Country == 'United States')
proxies['Port'] = proxies['Port'].astype(int)
proxies.sample(10)

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
286,41.75.4.208,53281,BW,Botswana,elite proxy,no,yes,21 minutes ago
6,200.236.221.242,33054,BR,Brazil,elite proxy,no,yes,1 minute ago
78,36.37.139.2,43997,KH,Cambodia,elite proxy,no,yes,4 minutes ago
278,124.41.243.22,47894,NP,Nepal,elite proxy,no,yes,21 minutes ago
276,195.138.82.198,40301,UA,Ukraine,elite proxy,no,yes,21 minutes ago
250,185.156.35.156,46579,PL,Poland,elite proxy,no,yes,20 minutes ago
2,103.106.114.134,8080,ID,Indonesia,elite proxy,no,yes,1 minute ago
8,87.197.156.62,23500,SK,Slovakia,elite proxy,no,yes,1 minute ago
17,169.255.126.211,30640,NG,Nigeria,elite proxy,no,yes,1 minute ago
50,125.27.251.24,36048,TH,Thailand,elite proxy,no,yes,2 minutes ago


In [4]:
# ## Your IP address
import requests
from bs4 import BeautifulSoup

URL = 'https://api.ipify.org?format=json'
req = requests.get(URL) # .json()
soup = BeautifulSoup(req.text, "lxml")
ip = soup.find('p').text
print('Your IP ', ip)

Your IP  {"ip":"73.142.126.75"}


In [1]:
## Test Headless Browser with BeautifulSoup
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = 'C:\ProgramData\Anaconda3\bin'

driver = webdriver.Chrome(executable_path='C:\ProgramData\Anaconda3\bin', options = options)
#driver.get("https://api.ipify.org/?format=json")
driver.get("http://www.opentable.com/boston-restaurant-listings")
html = driver.page_source
soup = BeautifulSoup(html)
#finding restaurant names
#this will create a list of html tags that match this criteria but it gives the results of only one page
for restaurant in soup.find_all(name='span', attrs={'class':'rest-row-name-text'}):
    print(restaurant.text)


WebDriverException: Message: 'Anaconda3in' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
#finding restaurant locations
for restaurant in soup.find_all(name='span', attrs={'class="rest-row-meta--location rest-row-meta-text'})

In [ ]:
## Unrendered proxy return without headless

import requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
URL = 'https://api.ipify.org?format=json'
proxy = {'https': '200.255.122.174:8080'}
req = requests.get(URL, proxies = proxy, headers = headers) # .json()
soup = BeautifulSoup(req.text, "lxml")
ip = soup.find('p').text
print('IP from scrape w/proxy ', ip)

# Output
# IP from scrape w/proxy  {"ip":"200.255.122.174"} # Tests proxy

In [ ]:
%%time

### Headless Scrape w/Single Proxy
### Imports list of https proxies and renders a sample in Chromium

## Import list of https proxies
import warnings;
warnings.filterwarnings('ignore')

import time
import os
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from bs4 import BeautifulSoup


def proxySelenium(proxyIP, proxyPort, headless = True):
    
    
    try:
        
        ourProxy = str(proxyIP) + ":" + str(proxyPort) # Use this for https anonymous/elite 

        proxy = Proxy({
            'proxyType': ProxyType.MANUAL,
            'httpProxy': ourProxy,
            'ftpProxy' : ourProxy,
            'sslProxy' : ourProxy,
            'noProxy'  : ''
        })

        options = webdriver.ChromeOptions()
        if headless == True:
          
          # Deletes web preview if it exists already
          if os.path.exists('data/' + str(proxyIP) + '_' + 'screenshot.png'):
            os.remove('data/' + str(proxyIP) + '_' + 'screenshot.png')
            
          options.add_argument('--headless')
          options.add_argument('--no-sandbox')
          options.add_argument('--disable-dev-shm-usage')
          options.binary_location = '/usr/bin/chromium-browser'

        # Configure capabilities 
        capabilities = dict(DesiredCapabilities.CHROME)
        proxy.add_to_capabilities(capabilities)

        driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver', desired_capabilities = capabilities, options = options)
        driver.set_page_load_timeout(200)

        # Open page and take screenshot
        driver.get("http://www.ipify.org/")

        if headless == False:
            time.sleep(3)

        driver.get_screenshot_as_file('data/' + str(proxyIP) + '_' + 'screenshot.png')

      # IP Text
        driver.get("https://api.ipify.org/?format=json")
        html = driver.page_source
        soup = BeautifulSoup(html)
        ip = soup.find('pre').text
        driver.quit()

        if headless == True:
          ## Show Screeenshot
          image = mpimg.imread('data/' + str(proxyIP) + '_' + 'screenshot.png')
          plt.figure(figsize=(20,20))
          plt.imshow(image)
          plt.show()
            
          # Remove Image
          os.remove('data/' + str(proxyIP) + '_' + 'screenshot.png')

        print("Working: " + str(proxyIP) + ":" + str(proxyPort))
        return "Working: " + str(proxyIP) + ":" + str(proxyPort)
      
    except:
        print("Proxy inactive: " + str(proxyIP) + ":" + str(proxyPort))
        return "Proxy inactive: " + str(proxyIP) + ":" + str(proxyPort)
        driver.quit()
    
# prox = proxies.sample(n=1)
proxyIP = '200.255.122.174'     # list(prox['IP Address'])[0] # 
proxyPort = '8080'              # list(prox['Port'])[0] 

print('IP to test: ', proxyIP, ' and Port:', proxyPort)
proxySelenium(proxyIP, proxyPort)

In [ ]:
%%time

### Parallelized proxy check over 10 sampled IPs w/Non-Headless

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def soupParallel(proxyIP, proxyPort):
    
    try:
        
        time.sleep(1)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
        URL = 'https://api.ipify.org?format=json'
        proxy = {'https': str(proxyIP) + ":" + str(proxyPort)}
        print("Input IP proxy: ", proxy)
        req = requests.get(URL, proxies = proxy, headers = headers) # .json()
        soup = BeautifulSoup(req.text, "lxml")
        ip = soup.find('p').text
        return 'IP from scrape w/proxy ' + str(ip)
        
    except:
        
        print("Proxy inactive: " + str(proxy))
        return "Proxy inactive: " + str(proxy)
        
## Parallel read of soupParallel function
from joblib import Parallel, delayed, parallel_backend
import multiprocessing

with parallel_backend('multiprocessing'):

  num_cores = multiprocessing.cpu_count()
  results = Parallel(n_jobs = num_cores)(delayed(soupParallel)(proxyIP = list(proxies['IP Address'])[i], 
        proxyPort = list(proxies['Port'])[i]) for i in range(0, len(proxies.head(10)) )) # len(proxies.head(10))

print(pd.DataFrame(results))

In [ ]:
%%time

### Headless Scrape w/Single Proxy
### Imports list of https proxies and renders a sample in Chromium

import warnings;
warnings.filterwarnings('ignore')
import time
import os
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from bs4 import BeautifulSoup


def proxySeleniumParallel(proxyIP, proxyPort, headless = True):
    
    try:
        
        ourProxy = str(proxyIP) + ":" + str(proxyPort) # Use this for https anonymous/elite 

        proxy = Proxy({
            'proxyType': ProxyType.MANUAL,
            'httpProxy': ourProxy,
            'ftpProxy' : ourProxy,
            'sslProxy' : ourProxy,
            'noProxy'  : ''
        })

        options = webdriver.ChromeOptions()
        if headless == True:
          
          # Deletes web preview if it exists already
          if os.path.exists('data/' + str(proxyIP) + '_' + 'screenshot.png'):
            os.remove('data/' + str(proxyIP) + '_' + 'screenshot.png')
            
          options.add_argument('--headless')
          options.add_argument('--no-sandbox')
          options.add_argument('--disable-dev-shm-usage')
          options.binary_location = '/usr/bin/chromium-browser'

        # Configure capabilities 
        capabilities = dict(DesiredCapabilities.CHROME)
        proxy.add_to_capabilities(capabilities)

        driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver', desired_capabilities = capabilities, options = options)
        driver.set_page_load_timeout(200)

        # Open page and take screenshot
        driver.get("https://www.whatismyip.com/ip-address-lookup/") # http://www.ipify.org/

        if headless == False:
            time.sleep(3)

        driver.get_screenshot_as_file('data/' + str(proxyIP) + '_' + 'screenshot.png')

      # IP Text
        driver.get("https://api.ipify.org/?format=json")
        html = driver.page_source
        soup = BeautifulSoup(html)
        ip = soup.find('pre').text
        driver.quit()

        if headless == True:
          ## Show Screeenshot
          image = mpimg.imread('data/' + str(proxyIP) + '_' + 'screenshot.png')
          plt.figure(figsize=(20,30))
          plt.imshow(image)
          plt.show()
            
          # Remove Image
          os.remove('data/' + str(proxyIP) + '_' + 'screenshot.png')

        print("Working: " + str(proxyIP) + ":" + str(proxyPort))
        return "Working: " + str(proxyIP) + ":" + str(proxyPort)
      
    except:
        print("Proxy inactive: " + str(proxyIP) + ":" + str(proxyPort))
        return "Proxy inactive: " + str(proxyIP) + ":" + str(proxyPort)
        driver.quit()
    

## Parallel read of soupParallel function
from joblib import Parallel, delayed, parallel_backend
import multiprocessing

with parallel_backend('multiprocessing'):
  num_cores = multiprocessing.cpu_count()
  results = Parallel(n_jobs = num_cores)(delayed(proxySeleniumParallel)(proxyIP =list(proxies['IP Address'])[i], 
        proxyPort = list(proxies['Port'])[i], headless = True) for i in range(len(proxies.head(10)))) # len(proxies.head(10))

print(pd.DataFrame(results))